Quick analysis

In [ ]:
#!/usr/bin/env python3
import os
import json
import h5py
import argparse
import pandas as pd
import numpy as np
import tinydb as db
from tinydb.storages import MemoryStorage
import matplotlib.pyplot as plt
# plt.style.use('../clint.mpl')
from matplotlib.colors import LogNorm

import boost_histogram as bh
import pickle as pl

from pygama import DataGroup
import pygama.io.lh5 as lh5
import pygama.analysis.histograms as pgh
import pygama.analysis.peak_fitting as pgf

In [ ]:
dg = DataGroup('cage.json', load=True)
run = 64
str_query = f'run=={run} and skip==False'
dg.file_keys.query(str_query, inplace=True)
view_cols = ['runtype', 'run', 'cycle', 'startTime', 'runtime', 'threshold']
print(dg.file_keys[view_cols])

In [ ]:
#get runtime, startime, runtype

runtype_list = np.array(dg.file_keys['runtype'])
runtype = runtype_list[0]

rt_min = dg.file_keys['runtime'].sum()
u_start = dg.file_keys.iloc[0]['startTime']
t_start = pd.to_datetime(u_start, unit='s') # str

In [ ]:
# get scan position

if runtype == 'alp':
    alphaDB = pd.read_hdf('alphaDB.h5')
    scan_pos = alphaDB.loc[alphaDB['run']==run]
    radius = np.array(scan_pos['radius'])[0]
    angle = np.array(scan_pos['angle'])[0]
    print(f'Radius: {radius}; Angle: {angle}')
# else:

In [ ]:
lh5_dir = dg.lh5_user_dir #if user else dg.lh5_dir
print(lh5_dir)
hit_list = lh5_dir + dg.file_keys['hit_path'] + '/' + dg.file_keys['hit_file']
print(hit_list)
df_hit = lh5.load_dfs(hit_list, ['trapEmax', 'bl','bl_sig','A_10','AoE', 'ts_sec', 'dcr_raw', 'dcr_ftp', 'dcr_max', 'tp_10', 'tp_90', 'tp_50', 'tp_80', 'tp_max'], 'ORSIS3302DecoderForEnergy/hit')
# df_hit = lh5.load_dfs(alpha_hit_list, ['trapEmax', 'bl','bl_sig','A_10','ts_sec', 'dcr_raw'], 'ORSIS3302DecoderForEnergy/hit')
# print(df_hit)
print(df_hit.columns)

dsc = df_hit[['bl','bl_sig','A_10','AoE', 'ts_sec', 'dcr_raw', 'dcr_ftp', 'dcr_max']].describe()
# dsc = df_hit[['bl','bl_sig','A_10','ts_sec', 'dcr_raw']].describe()
print(dsc)

In [ ]:
# get raw files for wfs

raw_lh5_dir = dg.lh5_dir
raw_list = raw_lh5_dir + dg.file_keys['raw_path'] + '/' + dg.file_keys['raw_file']

# print(raw_list)

# don't actually need this, just file list
# raw_df = lh5.load_dfs(raw_list, ['energy'], 'ORSIS3302DecoderForEnergy/raw')

dsp_config_file = ('./config_dsp.json')

In [ ]:
# Baseline vs E

etype = 'trapEmax'

elo, ehi, epb = 0, 50, 1
blo, bhi, bpb = 0, 10000, 100
nbx = int((ehi-elo)/epb)
nby = int((bhi-blo)/bpb)

h = plt.hist2d(df_hit['trapEmax'], df_hit['bl'], bins=[nbx,nby],
                       range=[[elo, ehi], [blo, bhi]], cmap='jet')

cb = plt.colorbar(h[3], ax=plt.gca())
plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('bl', ha='right', y=1)
plt.title('Baseline, alpha')
# plt.tight_layout()
plt.show()

In [ ]:
blo, bhi, bpb = 0, 10000, 100
nbx = int((bhi-blo)/bpb)

bl_hist, bins = np.histogram(df_hit['bl'], bins=nbx,
                range=[blo, bhi])

plt.semilogy(bins[1:], bl_hist, ds='steps', c='b', lw=1)

bl_cut_lo, bl_cut_hi = 8500,10000

plt.axvline(bl_cut_lo, c='r', lw=1)
plt.axvline(bl_cut_hi, c='r', lw=1)

plt.xlabel('bl', ha='right', x=1)
plt.ylabel('counts', ha='right', y=1)
plt.show()

In [ ]:
# use baseline cut
df_cut = df_hit.query('bl > 8500 and bl < 10000').copy()

# add new A/E column
df_cut['aoe'] = df_cut['A_10'] / df_cut['trapEmax']

#create dcr/E 
df_cut['dcr_norm'] = df_cut['dcr_raw'] / df_cut['trapEmax']

#create 10-90
df_cut['10-90'] = df_cut['tp_90']- df_cut['tp_10']

#create 50-90
df_cut['50-90'] = df_cut['tp_90']- df_cut['tp_50']

In [ ]:
# Make energy spectrum
elo, ehi, epb = 0, 35000, 10
# elo, ehi, epb = 0, 3000, 10
# elo, ehi, epb = 0, 6000, 10
        

nbx = int((ehi-elo)/epb)

energy_hist, bins = np.histogram(df_cut['trapEmax'], bins=nbx,
                range=[elo, ehi])

plt.semilogy(bins[1:], energy_hist, ds='steps', c='b', lw=1) #, label=f'{etype}'

plt.xlabel(etype, ha='right', x=1)
plt.ylabel('cts / sec', ha='right', y=1)
# plt.legend()
plt.title(f'{etype}: {runtype} run {run}; {rt_min:.2f} mins')
# plt.savefig(f'./plots/normScan/{runtype}_energy_run{run}.pdf', dpi=200)
plt.show()

In [ ]:
# Alpha A/E
alo, ahi, apb = 0.0, 0.09, 0.0001
elo, ehi, epb = 0, 25000, 10
# elo, ehi, epb = 0, 3000, 10
# elo, ehi, epb = 0, 6000, 10
        

nbx = int((ehi-elo)/epb)
nby = int((ahi-alo)/apb)

h = plt.hist2d(df_cut['trapEmax'], df_cut['AoE'], bins=[nbx,nby],
                range=[[elo, ehi], [alo, ahi]], cmap='jet', norm=LogNorm())

plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('A/E', ha='right', y=1)
# plt.legend()
plt.title(f'A/E: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_a10oE_run{run}.png', dpi=200)
plt.show()

In [ ]:
# DCR vs E
etype = 'trapEmax'
#       elo, ehi, epb = 0, 6000, 10
elo, ehi, epb = 0, 25000, 10
# dlo, dhi, dpb = -0.08, 0.025, 0.0005
dlo, dhi, dpb = -0.1, 0.025, 0.0005

nbx = int((ehi-elo)/epb)
nby = int((dhi-dlo)/dpb)

alpha_dcr_hist = plt.hist2d(df_cut['trapEmax'], df_cut['dcr_norm'], bins=[nbx,nby],
                range=[[elo, ehi], [dlo, dhi]], cmap='jet', norm=LogNorm())

plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('DCR_norm', ha='right', y=1)
plt.title(f'DCR: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_dcr_norm_run{run}.png', dpi=200)
plt.show()

In [ ]:
#creat new DCR
const = 0.0555
df_cut['dcr_linoff'] = df_cut['dcr_raw'] + const*df_cut['trapEmax']

In [ ]:
old_dcr_cut = df_cut.query('trapEmax > 5000 and trapEmax < 15000 and dcr_norm > -0.0515 and dcr_norm < -0.02').copy()
len(old_dcr_cut)

In [ ]:
# DCR vs E
etype = 'trapEmax'
#       elo, ehi, epb = 0, 6000, 10
elo, ehi, epb = 0, 25000, 10
# dlo, dhi, dpb = -0.08, 0.025, 0.0005
# dlo, dhi, dpb = -0.1, 0.025, 0.0005

dlo, dhi, dpb = -100, 300, 0.6

nbx = int((ehi-elo)/epb)
nby = int((dhi-dlo)/dpb)

alpha_dcr_hist = plt.hist2d(df_cut['trapEmax'], df_cut['dcr_linoff'], bins=[nbx,nby],
                range=[[elo, ehi], [dlo, dhi]], cmap='jet', norm=LogNorm()) #

plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('DCR_linoffset', ha='right', y=1)
plt.title(f'DCR: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_dcr_linoffset_run{run}.png', dpi=200)
plt.show()

In [ ]:
cut = (df_cut['dcr_linoff']>50) & (df_cut['dcr_linoff']<200) & (df_cut['trapEmax']>5000) & (df_cut['trapEmax']<12000)
cut2 = (df_cut['trapEmax']>5000) & (df_cut['trapEmax']<10000)

In [ ]:
from pygama.io.WaveformBrowser import WaveformBrowser

In [ ]:
# Now set up browsing waveforms
browser = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file,
                          waveforms='wf_blsub', # baseline subtracted waveform
                          legend=['trapEmax', 'AoE'], # build a legend
                          align='tp_0', # align waveforms along t0
                          x_lim=(-500, 80000), # x axis range
                          norm='trapEmax', # normalize by energy
                          selection = cut, # use our mask from before to draw MS wfs
                          n_drawn = 8, # Draw 8 at a time
                         )

In [ ]:
# Now set up browsing waveforms
browser2 = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file,
                          waveforms='wf_blsub', # baseline subtracted waveform
                          legend=['trapEmax', 'AoE'], # build a legend
                          align='tp_0', # align waveforms along t0
                          x_lim=(-500, 80000), # x axis range
                          norm='trapEmax', # normalize by energy
                          selection = cut2, # use our mask from before to draw MS wfs
                          n_drawn = 8, # Draw 8 at a time
                         )

In [ ]:
# Draw first waveform in file
# browser.draw_entry()

# Draw next 8 waveforms (from n_drawn)
# browser.new_figure()
browser.draw_next()
browser2.ax = browser.ax
browser2.fig = browser.fig
browser2.draw_next()

In [ ]:
# Draw first waveform in file
# browser.draw_entry()

# Draw next 8 waveforms (from n_drawn)
# browser.new_figure()
# browser.draw_next()
browser2.ax.lines[0]
browser2.draw_next()


In [ ]:
# Draw first waveform in file
# browser.draw_entry()

# Draw next 8 waveforms (from n_drawn)
# browser.new_figure()
browser2.draw_next()

In [ ]:
#Draw waveforms as a slideshow. Wheeeee!
browser.new_figure()
for _ in zip(browser, range(10)):
    browser.new_figure()

In [ ]:
# DCR 1D hist
etype = 'trapEmax'
#       elo, ehi, epb = 0, 6000, 10
elo, ehi, epb = 0, 15000, 10
# dlo, dhi, dpb = -0.08, 0.025, 0.0005
# dlo, dhi, dpb = -0.1, 0.025, 0.0005

# dlo, dhi, dpb = -20, 20, 1
dlo, dhi, dpb = -200, 300, 0.6

nbx = int((dhi-dlo)/dpb)

dcr_hist, bins = np.histogram(df_cut['dcr_linoff'], bins=nbx,
                range=[dlo, dhi])

plt.semilogy(bins[1:], dcr_hist, ds='steps', c='b', lw=1) #, label=f'{etype}'



# plt.legend()
plt.xlabel('DCR_linoffset', ha='right', y=1)
plt.ylabel('cts / sec', ha='right', y=1)
plt.title(f'DCR: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_1d_dcr_linoffset_run{run}.png', dpi=200)
plt.show()

In [ ]:
# DCR vs A/E
etype = 'trapEmax'
#       elo, ehi, epb = 0, 6000, 10
elo, ehi, epb = 0, 25000, 10
# dlo, dhi, dpb = -0.08, 0.025, 0.0005
# dlo, dhi, dpb = -0.1, 0.025, 0.0005

alo, ahi, apb = 0.0, 0.09, 0.0001
# dlo, dhi, dpb = -100, 300, 0.6
dlo, dhi, dpb = 15, 300, 0.6

nbx = int((ahi-alo)/apb)
nby = int((dhi-dlo)/dpb)

alpha_dcr_hist = plt.hist2d(df_cut['AoE'], df_cut['dcr_linoff'], bins=[nbx,nby],
                range=[[alo, ahi], [dlo, dhi]], cmap='jet', norm=LogNorm()) #

plt.xlabel('A/E', ha='right', x=1)
plt.ylabel('DCR_linoffset', ha='right', y=1)
plt.title(f'DCR vs A/E: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_a10oE_vs_dcr_linoffset_run{run}.png', dpi=200)
plt.show()

In [ ]:
# alpha_cut = 'dcr_linoff > 25 and dcr_linoff < 200 and AoE >0.04 and AoE < 0.05 and trapEmax < 15000'
alpha_cut = 'trapEmax < 12000 and dcr_linoff > 15 or dcr_linoff < -10'
new_dcr_cut = df_cut.query(alpha_cut).copy()
len(new_dcr_cut)

In [ ]:
# DCR vs A/E
etype = 'trapEmax'
#       elo, ehi, epb = 0, 6000, 10
elo, ehi, epb = 0, 25000, 10
# dlo, dhi, dpb = -0.08, 0.025, 0.0005
# dlo, dhi, dpb = -0.1, 0.025, 0.0005

alo, ahi, apb = 0.0, 0.09, 0.0001
dlo, dhi, dpb = -100, 300, 0.6

nbx = int((ahi-alo)/apb)
nby = int((dhi-dlo)/dpb)

alpha_dcr_hist = plt.hist2d(new_dcr_cut['aoe'], new_dcr_cut['dcr_linoff'], bins=[nbx,nby],
                range=[[alo, ahi], [dlo, dhi]], cmap='jet', norm=LogNorm()) #

plt.xlabel('A/E', ha='right', x=1)
plt.ylabel('DCR_linoffset', ha='right', y=1)
plt.title(f'DCR vs A/E after alpha cut: {runtype} run {run}, {rt_min:.2f} mins \n cut: {alpha_cut} \n')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_alphaCut_a10oE_vs_dcr_linoffset_run{run}.png', dpi=200)
plt.show()

In [ ]:
# Alpha A/E
alo, ahi, apb = 0.0, 0.09, 0.0001
# elo, ehi, epb = 0, 25000, 10
# elo, ehi, epb = 0, 3000, 10
elo, ehi, epb = 0, 6000, 10
        

nbx = int((ehi-elo)/epb)
nby = int((ahi-alo)/apb)

h = plt.hist2d(new_dcr_cut['trapEmax'], new_dcr_cut['aoe'], bins=[nbx,nby],
                range=[[elo, ehi], [alo, ahi]], cmap='jet', norm=LogNorm())

plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('A/E', ha='right', y=1)
# plt.legend()
plt.title(f'A/E after cut: {runtype} run {run}, {rt_min:.2f} mins \n cut: {alpha_cut} \n')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_alphaCut_a10oE_run{run}.png', dpi=200)
plt.show()

In [ ]:
# alpha_cut = 'dcr_linoff > 25 and dcr_linoff < 200 and AoE >0.04 and AoE < 0.05 and trapEmax < 15000'
aoe_cut = 'trapEmax > 2000 and trapEmax < 12000 and AoE >0.03 and AoE < 0.045'
aoe_df = df_cut.query(aoe_cut).copy()
len(aoe_df)

In [ ]:
# DCR vs E
etype = 'trapEmax'
#       elo, ehi, epb = 0, 6000, 10
elo, ehi, epb = 0, 25000, 10
# dlo, dhi, dpb = -0.08, 0.025, 0.0005
# dlo, dhi, dpb = -0.1, 0.025, 0.0005

dlo, dhi, dpb = -100, 300, 0.6

nbx = int((ehi-elo)/epb)
nby = int((dhi-dlo)/dpb)

alpha_dcr_hist = plt.hist2d(aoe_df['trapEmax'], aoe_df['dcr_linoff'], bins=[nbx,nby],
                range=[[elo, ehi], [dlo, dhi]], cmap='jet', norm=LogNorm()) #

plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('DCR_linoffset', ha='right', y=1)
plt.title(f'DCR: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_dcr_linoffset_run{run}.png', dpi=200)
plt.show()

In [ ]:
# 1d A/E after cut
alo, ahi, apb = 0.0, 0.09, 0.0001

nbx = int((ahi-alo)/apb)

aoe_hist, bins = np.histogram(df_cut['aoe'], bins=nbx,
                range=[alo, ahi])

plt.semilogy(bins[1:], aoe_hist, ds='steps', c='b', lw=1) #, label=f'{etype}'



# plt.legend()
plt.xlabel('A/E', ha='right', y=1)
plt.ylabel('cts / sec', ha='right', y=1)
plt.title(f'A/E before cut: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_1d_aoe_run{run}.png', dpi=200)
plt.show()

In [ ]:
# 1d A/E after cut
alo, ahi, apb = 0.0, 0.09, 0.0001

nbx = int((ahi-alo)/apb)

cut_aoe_hist, bins = np.histogram(new_dcr_cut['aoe'], bins=nbx,
                range=[alo, ahi])

plt.semilogy(bins[1:], cut_aoe_hist, ds='steps', c='b', lw=1) #, label=f'{etype}'



# plt.legend()
plt.xlabel('A/E', ha='right', y=1)
plt.ylabel('cts / sec', ha='right', y=1)
plt.title(f'A/E after cut: {runtype} run {run}, {rt_min:.2f} mins\n cut: {alpha_cut} \n')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_alphaCut_1d_aoe_run{run}.png', dpi=200)
plt.show()

In [ ]:
aoe_cut = new_dcr_cut.query('aoe < 0.05').copy()
len(aoe_cut)

In [ ]:
# DCR vs E
etype = 'trapEmax'
#       elo, ehi, epb = 0, 6000, 10
elo, ehi, epb = 0, 15000, 10
# dlo, dhi, dpb = -0.08, 0.025, 0.0005
# dlo, dhi, dpb = -0.1, 0.025, 0.0005

# dlo, dhi, dpb = -20, 20, 1
dlo, dhi, dpb = -200, 300, 0.6

nbx = int((dhi-dlo)/dpb)

dcr_hist, bins = np.histogram(aoe_cut['dcr_linoff'], bins=nbx,
                range=[dlo, dhi])

plt.semilogy(bins[1:], dcr_hist, ds='steps', c='b', lw=1) #, label=f'{etype}'



# plt.legend()
plt.xlabel('DCR_linoffset', ha='right', y=1)
plt.ylabel('cts / sec', ha='right', y=1)
plt.title(f'DCR after A/E cut: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_dcr_norm_run{run}.png', dpi=200)
plt.show()

In [ ]:
# DCR vs E
etype = 'trapEmax'
#       elo, ehi, epb = 0, 6000, 10
elo, ehi, epb = 0, 35000, 10
# dlo, dhi, dpb = -0.08, 0.025, 0.0005
# dlo, dhi, dpb = -0.1, 0.025, 0.0005

dlo, dhi, dpb = -20, 300, 0.6

nbx = int((ehi-elo)/epb)
nby = int((dhi-dlo)/dpb)

alpha_dcr_hist = plt.hist2d(old_dcr_cut['trapEmax'], old_dcr_cut['dcr_linoff'], bins=[nbx,nby],
                range=[[elo, ehi], [dlo, dhi]], cmap='jet', norm=LogNorm()) #

plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('DCR_linoffset', ha='right', y=1)
plt.title(f'Old DCR cut: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_dcr_linoffset_run{run}.png', dpi=200)
plt.show()

In [ ]:
# DCR vs E
etype = 'trapEmax'
#       elo, ehi, epb = 0, 6000, 10
elo, ehi, epb = 0, 35000, 10
# dlo, dhi, dpb = -0.08, 0.025, 0.0005
# dlo, dhi, dpb = -0.1, 0.025, 0.0005

dlo, dhi, dpb = -20, 300, 0.6

nbx = int((ehi-elo)/epb)
nby = int((dhi-dlo)/dpb)

alpha_dcr_hist = plt.hist2d(new_dcr_cut['trapEmax'], new_dcr_cut['dcr_linoff'], bins=[nbx,nby],
                range=[[elo, ehi], [dlo, dhi]], cmap='jet', norm=LogNorm()) #

plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('DCR_linoffset', ha='right', y=1)
plt.title(f'New DCR cut: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_dcr_linoffset_run{run}.png', dpi=200)
plt.show()